## GenericsPrediction - Data Munging

###Contact: Adrian Lam; ayplam@gmail.com

#### Notes:
* Download the [NDC Drug Database](http://www.fda.gov/Drugs/InformationOnDrugs/ucm142438.htm) and unzip
* Download the [Orange Book of Therapeautic Equivalance](http://www.fda.gov/downloads/Drugs/DevelopmentApprovalProcess/UCM071436.pdf)
* Ensure that all downloaded files are in the same directory as the ipython notebook


In [1]:
import csv
import time
%matplotlib inline
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns
import glob
import re
import pandas as pd

In [9]:
# Function to parse "product.txt" from the FDA NDC Directory

def nonprop_parse(nonproprietaryname):
    
    words_to_remove_nonprop = ['TABLET','TABLETS','MG','AND','\.','/','\d+','"',"'",'\s+']
    
    re.sub('HCL','HYDROCHLORIDE',line[5])

    # Words to remove from nonproprietary name field
    for word in words_to_remove_nonprop:
        nonproprietaryname = re.sub(word,' ',nonproprietaryname)
        
    # Remove trailing spaces
    while nonproprietaryname[-1] is ' ':
        nonproprietaryname = nonproprietaryname[:-1]
    
    return nonproprietaryname

def supplier_parse(supplier):
    
    store = supplier
    
    # replace period with space.
    supplier = re.sub('\.',' ',supplier)
    
    words_to_remove_supplier = ['\.','INC','LLC','\?',"'",'(\s+)?\&(\s+)?','\-', \
                                'LIMITED',' RX ', 'OF','\(','\)','AND',r'\\','/']
    
    for word in words_to_remove_supplier:
        supplier = re.sub(word, ' ',supplier)
        
    supplier = re.sub('\+','PLUS',supplier)
    
    # Remove all extra spaces
    supplier = re.sub('\s+',' ',supplier)
    
    # Fixes most misspellings of "PHARMACEUTICALS" (Will rename one supplier from "PHARMACEUTICA" but that's okay)
    supplier = re.sub('[PHARMCEUTILS]{13,18}','PHARMACEUTICALS',supplier)
    
    match = re.match('^(?P<char1>\w)\s+(?P<char2>\w\s+)',supplier)
    if match:
        supplier = re.sub(match.group(0), match.group('char1') + match.group('char2'), supplier)    

    # Get the first two words in the supplier. 
    match = re.match('^(\s+)?(?P<supplier>\w+(\s+\w+)?)',supplier)
    
    if match:
        if len(match.group('supplier')) < 2:
            print 'WARNING(supplier_parse): supplier name fewer than two words: ', store, supplier
        return match.group('supplier')
    else:
        print store, supplier
    



### Parsing product.txt
* Splits the pharmaceutical class into "Established Pharmaceutical Class (EPC)" and "Mechanism of Action (MOA)"
* Cleans up the nonproprietaryname field to allow for better grouping in SQL
* Cleans up the supplier name field to allow for better grouping in SQL

#### The final print statement is a check to determine how large the SQL table fields need to be upon creation.

In [10]:
f = open('product.txt','rb')
fname = "product_parsed.csv" 
g = open(fname,'wb')

line = f.readline()
header = line
print header
num_fields = len(line.strip().split("\t"))
print num_fields

maxes = [0] * (num_fields+3)


while True:
# for i in xrange(20):
    
    line = f.readline()
    
    if line == '':
        break
    
    line = line.strip()
    line = line.upper()
    # Turn all commas into spaces (comma is my psql delimiter)
    line = re.sub(',',' ',line)
    # Turn tabs into commas. This makes things easier to upload into PSQL
    line = re.sub('\t',',',line)
    # Add NULLs so TIMESTAMPS read in properly in SQL
    line = re.sub(',,',',NULL,',line)
    line = re.sub(',,',',NULL,',line)
    line = re.sub(',,',',NULL,',line)
    
    # Need to split the line before you can start parsing on the field.
    line = line.split(',')
        
    ## Parse nonproprietary name for more uniformity.
    line[5] = nonprop_parse(line[5])
    line[12] = supplier_parse(line[12])
    
    # Join the line into a single string
    line = ','.join(line)
   
    while len(line.split(',')) < num_fields:
        line += ',NULL'
    
    ## This section parses the pharmaceutical class into separate fields, EPC and MOA
    # Separate the pharmaceutical class into EPC and MOA. 
    tmp = line.split(',')
    epc = ';'.join(re.findall(r'[A-Za-z0-9_ ]+\[EPC\]',tmp[-2]))
    epc = re.sub('\s+',' ',re.sub(r'\[EPC\]','',epc))
    moa = ';'.join(re.findall(r'[A-Za-z0-9_ ]+\[MOA\]',tmp[-2]))
    moa = re.sub('\s+',' ',re.sub(r'\[MOA\]','',moa))
    pe = ' '.join(re.findall(r'[A-Za-z0-9_ ]+\[PE\]',tmp[-2]))        
    
    if epc:
        line += ', '
        line += epc
    else:
        line += ',NULL'
        
    if moa:
        line += ', '
        line += moa
    else:
        line += ',NULL'
    
    # This checks the length of each field for SQL tables.
    lengthcheck = line.split(',')
    for num,item in enumerate(lengthcheck):
        maxes[num] = max([maxes[num], len(item)])
        
    g.write(line + " \n")

g.close()

for num,item in enumerate(header.split('\t')):
    print item, maxes[num]

PRODUCTID	PRODUCTNDC	PRODUCTTYPENAME	PROPRIETARYNAME	PROPRIETARYNAMESUFFIX	NONPROPRIETARYNAME	DOSAGEFORMNAME	ROUTENAME	STARTMARKETINGDATE	ENDMARKETINGDATE	MARKETINGCATEGORYNAME	APPLICATIONNUMBER	LABELERNAME	SUBSTANCENAME	ACTIVE_NUMERATOR_STRENGTH	ACTIVE_INGRED_UNIT	PHARM_CLASSES	DEASCHEDULE

18
PRODUCTID 47
PRODUCTNDC 10
PRODUCTTYPENAME 27
PROPRIETARYNAME 226
PROPRIETARYNAMESUFFIX 126
NONPROPRIETARYNAME 497
DOSAGEFORMNAME 48
ROUTENAME 118
STARTMARKETINGDATE 8
ENDMARKETINGDATE 8
MARKETINGCATEGORYNAME 40
APPLICATIONNUMBER 11
LABELERNAME 29
SUBSTANCENAME 3814
ACTIVE_NUMERATOR_STRENGTH 742
ACTIVE_INGRED_UNIT 2055
PHARM_CLASSES 3998
DEASCHEDULE
4


### Parsing the Orange Book of Therapeutic Equivalences
* Data from: http://www.fda.gov/downloads/Drugs/DevelopmentApprovalProcess/UCM071436.pdf
* Linux command for conversion of pdf to txt: pdftotext -layout -htmlmeta UCM071436.pdf
* Parse the resulting file here
* (HTML file for python parsing is included; pdftotext command is optional if not running Linux)

In [5]:
fname = 'UCM071436.html'
f = open(fname,'rb')

drug_output = []

andalist = []
df_ndapairs = pd.DataFrame({'drug':[],'appnum':[]})

# This is where the listing of all the drugs begins
while True:
    line = f.readline()
    if '35TH EDITION - 2015 - APPROVED DRUG PRODUCT LIST' in line:
        break;

s_drugname = []
s_appnum = []
s_dosage = []

while True:
    
    line = f.readline()
    
    if line == '':
        break;
    
    # Ongoing 
    if line[:3].isalpha():
        drugname = line.strip()
        
    if line[:2] == '  ' and line[3].isalpha():
        dosage = line.strip()
        dosage = re.sub(';',' ',dosage)
    
    if re.findall('[AN]\d{6,6}', line) and line[:2] == 'AB':
        drug_output.append(line.strip())
        
        appnum = re.findall('[AN]\d{6,6}', line)
        appnum = appnum[0]
        
        if appnum[0] == 'A':
            appnum = re.sub('A','ANDA',appnum)
        elif appnum[0] == 'N':
            appnum = re.sub('N','NDA',appnum)
        
        s_appnum.append(appnum)
        s_drugname.append(drugname + ' ' + dosage)
        s_dosage.append(dosage)

    
df_appnum = pd.DataFrame({'drug':s_drugname, 'appnum':s_appnum})


for drug in df_appnum['drug'].unique():
    if not df_appnum[df_appnum['drug'] == drug]['appnum'].str.contains('^ANDA').any() and df_appnum[df_appnum['drug'] == drug]['appnum'].str.contains('^NDA').any():
        df_appnum = df_appnum.drop(df_appnum[df_appnum['drug'] == drug].index)


df_appnum.index = xrange(0,len(df_appnum))
df_appnum.to_csv('/home/vagrant/datacourse/user/mck/df_ndapairs.csv',sep=',',index=False)


